In [1]:
import pandas as pd
df = pd.read_csv("merged_df")

In [2]:
from tqdm import tqdm
import pandas as pd
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, classification_report

# Assuming df is your DataFrame
# Replace 'target_variable' with your actual target variable
X = df[['soc','sum_current', 'median_current', 'iqr_current', 'mode_volt', 'variance_current', 'kurtosis_current', 'median_volt', 'mean_volt',  'mean_current', 'max_temp', 'min_temp', 'std_current']]

y = df['label']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize RandomOverSampler
oversampler = RandomOverSampler(random_state=42)

# Fit and transform the training data
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)

# Check the class distribution after oversampling
print(pd.Series(y_train_resampled).value_counts())

10    6726687
0     6726687
Name: label, dtype: int64


In [3]:
# df.drop(['car','charge_segment'],axis=1,inplace=True)
# Initialize Decision Tree classifier (base estimator)
base_classifier = DecisionTreeClassifier(max_depth=3)  # Adjust max_depth as needed

# Initialize Bagging classifier
bagging_model = BaggingClassifier(base_classifier, n_estimators=50, random_state=42, verbose = 1)  # Adjust n_estimators as needed

# Train the model
for _ in tqdm(range(10)):
    bagging_model.fit(X_train_resampled, y_train_resampled)

# Make predictions on the test set
y_pred_bagging = bagging_model.predict(X_test)

# Evaluate the model
accuracy_bagging = accuracy_score(y_test, y_pred_bagging)
print(f'Bagging Decision Tree Accuracy: {accuracy_bagging}')
print('Classification Report:')
print(classification_report(y_test, y_pred_bagging))

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_bagging)
print('Confusion Matrix:')
print(conf_matrix)


100%|██████████| 10/10 [7:47:14<00:00, 2803.46s/it] 


Bagging Decision Tree Accuracy: 0.7842694402680503
Classification Report:
              precision    recall  f1-score   support

           0       0.93      0.79      0.85   2884189
          10       0.49      0.78      0.60    759305

    accuracy                           0.78   3643494
   macro avg       0.71      0.78      0.73   3643494
weighted avg       0.84      0.78      0.80   3643494

Confusion Matrix:
[[2267789  616400]
 [ 169613  589692]]
